In [ ]:
!pip install scikit-learn
!pip install numpy==1.16.1

In [ ]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
np.random.seed(12227)

In [ ]:
def DataPreparation(data_input_file):
  print('JIANGUANG HUANG et al. 2019 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
 
  return X,Y,folds

In [ ]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [ ]:
def Run_JIANGUANG_HUANG(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)

  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [ ]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]  
  _,img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
  
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=build_model(img_rows,img_cols,n_class)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='ADAM')
    model.fit(X_train, y[train_idx], batch_size=192, epochs=200,verbose=1)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [ ]:
tf.keras.backend.set_image_data_format('channels_last')

In [ ]:

def build_model(row,col,num_classes):

  
 
  layers = [
    
       tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
     
     
        
        tf.keras.layers.LSTM(64,return_sequences=True,  input_shape=(row, col)),
        tf.keras.layers.LSTM(64,return_sequences=True), 
        tf.keras.layers.Conv1D(filters=64,kernel_size=5,strides=2),
        tf.keras.layers.MaxPool1D(pool_size=2,strides = 2),
        tf.keras.layers.Conv1D(filters=128,kernel_size= 3,strides=1),
        tf.keras.layers.GlobalAveragePooling1D(),        
        tf.keras.layers.BatchNormalization(),  
                       

       
        
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model